## Utility Maximization

### Job-Hopping and Wages-Utility-Maximization:      
You are a worker who starts every day either employed or unemployed. If you start your day employed, you work on your job for the day (one of $n$ jobs, as elaborated later) and you get to earn the wage of the job for the day. However, at the end of the day, you could lose your job with probability $\alpha \in [0,1]$, in which case you start the next day unemployed. If at the end of the day, you do not lose your job (with probability $1-\alpha$), then you will start the next day with the same job (and hence, the same daily wage). On the other hand, if you start your day unemployed, then you will be randomly offered one of $n$ jobs with daily wages $w_1, w_2, \ldots w_n \in \mathbb{R}^+$ with respective job-offer probabilities $p_1, p_2, \ldots p_n \in [0,1]$ (with $\sum_{i=1}^n p_i = 1$). You can choose to either accept or decline the offered job. If you accept the job-offer, your day progresses exactly like the {\em employed-day} described above (earning the day's job wage and possibly (with probability $\alpha$) losing the job at the end of the day). However, if you decline the job-offer, you spend the day unemployed, receive the unemployment wage $w_0 \in \mathbb{R}^+$ for the day, and start the next day unemployed. The problem is to identify the optimal choice of accepting or rejecting any of the job-offers the worker receives, in a manner that maximizes the infinite-horizon {\em Expected Discounted-Sum of Wages Utility}. Assume the daily discount factor for wages (employed or unemployed) is $\gamma \in [0,1)$. Assume Wages Utility function to be $U(w) = \log(w)$ for any wage amount $w \in \mathbb{R}^+$. So you are looking to maximize 
$$
\mathbb{E}[\Sigma_{u=t}^\infty \gamma^{u-t} \cdot \log(w_{i_u})]
$$ 
at the start of a given day $t$ ($w_{i_u}$ is the wage earned on day $u$, $0\leq i_u \leq n$ for all $u\geq t$).

In [1]:
import numpy as np

#### Question 1:      
Express with clear mathematical notation the state space, action space, transition function, reward function, and write the Bellman Optimality Equation customized for this MDP.

The state space is the set $\mathcal{S} = \left\{ 0, 1, 2, ..., n\right\}$, where 0 = the unemployed state, and $n$ = number of jobs available.         
The action space is the set $\mathcal{A} = \left\{ \text{A}, \text{R}, \text{W} \right\}$, where:
- A = "accept the job offer"
- R = "reject the job offer"
- W = "work" (when you don't need to take an action as you're not laid off)


The transitions are:
$$
\begin{aligned}
    \mathbb{P}(S_t = i, A_t = W, S_{t + 1} = i) & = 1 - \alpha \\ 
    \mathbb{P}(S_t = i, A_t = W, S_{t + 1} = 0) & = \alpha \\
    \mathbb{P}(S_t = 0, A_t = A, S_{t + 1} = i) & = 1 - \alpha \\
    \mathbb{P}(S_t = 0, A_t = A, S_{t + 1} = 0) & = \alpha
\end{aligned}
$$
where $i$ is the job $i$ being offered when you're unemployed.

The rewards are:
$$
\begin{aligned}
    \mathcal{R}(S_t = i, A_t = W) & = \log \left( w_i \right) \\  
    \mathcal{R}(S_t = 0, A_t = A) & = \log \left( w_i \right) \\
    \mathcal{R}(S_t = 0, A_t = R) & = \log \left( w_0 \right)
\end{aligned}
$$
where $w_{i_u}$ is the wage earned per day from working job $i$ on day $u$ and $w_0$ is the unemployment wage.

#### Question 2:       
You can solve this Bellman Optimality Equation (hence, solve for the Optimal Value Function and the Optimal Policy) with a numerical iterative algorithm (essentially a Dynamic Programming algorithm customized to this problem). Write Python code for this numerical algorithm. Clearly define the inputs and outputs of your algorithm with their types (int, float, List, Mapping etc.). For this problem, don't use any of the MDP/DP code from the git repo, write this customized algorithm from scratch.

The Bellman optimality equations become:
$$
\begin{aligned}
V_i^* & = \log \left( w_i \right) + \gamma * \left[ \alpha \cdot V_0^* + (1 - \alpha) * V_i^* \right] \\ 
\therefore V_i^* & = \frac{\log \left( w_i \right) + \gamma \cdot \alpha \cdot V_0^*}{1 - \gamma * (1 - \alpha)} \\ \\ 

Q(0, A) & = \log \left( w_i \right) + \gamma * \left[ \alpha \cdot V_0^* + (1 - \alpha) \cdot V_i ^* \right] \\
Q(0, R) & = \log \left( w_0 \right) + \gamma * 1 \cdot V_0^* \\ 
\therefore V_0^* & = \Sigma_{i = 1}^n p_i \cdot \max \left\{ \log \left( w_i \right) + \gamma * \left[ \alpha \cdot V_0^* + (1 - \alpha) \cdot V_i ^* \right], \log \left( w_0 \right) + \gamma * 1 \cdot V_0^* \right\}
\end{aligned}
$$

In [2]:
num_jobs = 3
wages = np.array([10.0, 9.0, 5.0])
unemployment_wage = 8.0
job_offer_probs = np.array([0.3, 0.5, 0.2])
alpha = 0.1
gamma = 0.9

In [3]:
v_0 = np.zeros(num_jobs + 1)
TOLERANCE = 1e-5
MAX_ITER = 1_000
num_iter = 0

while num_iter < MAX_ITER:
    v_1 = np.zeros(num_jobs + 1)
    v_1[1:] = (np.log(wages) + gamma * alpha * v_0[0]) / (1.0 - gamma * (1.0 - alpha))

    q_accept = np.log(wages) + gamma * (alpha * v_0[0] + (1.0 - alpha) * v_1[1:])
    q_reject = np.log(unemployment_wage) + gamma * v_0[0]

    # calculate expected future utilities from accepting/rejecting jobs
    v_1[0] = np.sum(job_offer_probs * np.maximum(q_accept, q_reject))

    if np.max(np.abs(v_1 - v_0)) < TOLERANCE:
        v_0 = v_1
        break

    v_0 = v_1
    num_iter += 1


optimal_vf = v_1
optimal_vf

array([22.34041222, 22.7011649 , 22.14663587, 19.05302185])

### What is the optimal policy?

In [4]:
q_accept_star = np.log(wages) + gamma * (
    alpha * optimal_vf[0] + (1.0 - alpha) * optimal_vf[1:]
)
q_reject_star = np.log(unemployment_wage) + gamma * optimal_vf[0]

optimal_policy = q_accept_star > q_reject_star

# print out policy from the s = 0 state
for i in range(num_jobs):
    print(
        f"Job {i + 1}:\n\tWages = ${wages[i]:.2f}\n\t"
        f"Expected rewards of accepting = ${q_accept_star[i]:.3f}\n\t"
        f"Expected rewards of rejecting = ${q_reject_star:.3f}\n\t"
        f"Decision = {'Accept' if optimal_policy[i] else 'Reject'}\n"
    )

Job 1:
	Wages = $10.00
	Expected rewards of accepting = $22.701
	Expected rewards of rejecting = $22.186
	Decision = Accept

Job 2:
	Wages = $9.00
	Expected rewards of accepting = $22.147
	Expected rewards of rejecting = $22.186
	Decision = Reject

Job 3:
	Wages = $5.00
	Expected rewards of accepting = $19.053
	Expected rewards of rejecting = $22.186
	Decision = Reject

